In [57]:
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [2]:
data = pd.read_csv("flights.csv")

In [3]:
data.head()

,Month,Day,Day Of Week,Airline,Origin Airport Delay Rate,Destination Airport Delay Rate,Scheduled Time,Distance,Scheduled Departure Hour,Scheduled Departure Minute,Arrival Delay
0,1,2,5,WN,0.000000,0.000000,225,1390,8,55,-13.0
1,1,4,7,AA,0.000000,0.590909,172,1096,8,30,-12.0
2,1,4,7,US,0.000000,0.000000,46,130,0,30,189.0
3,1,3,6,WN,0.000000,0.000000,160,925,9,25,-7.0
4,1,5,1,EV,0.633333,0.000000,69,113,5,46,-4.0


In [22]:
data_ohe = pd.get_dummies(data, drop_first=True, dtype = int)

In [27]:
target = data_ohe["Arrival Delay"]
features = data_ohe.drop("Arrival Delay", axis = 1)
features_train, features_valid, target_train, target_valid = train_test_split(features,target, test_size=0.25, random_state=12345)


In [29]:
numeric = ['Day', 'Day Of Week', 'Origin Airport Delay Rate','Destination Airport Delay Rate', 'Scheduled Time', 'Distance','Scheduled Departure Hour', 'Scheduled Departure Minute']

In [30]:
scaler = StandardScaler().fit(features_train[numeric])

In [36]:
features_train[numeric]= scaler.transform(features_train[numeric])
features_valid[numeric]= scaler.transform(features_valid[numeric])
print(features_train.shape)
print(features_valid.shape)

(58431, 22)
(19478, 22)


In [37]:
df = pd.read_csv("flights_preprocessed.csv")

In [44]:
target = df['Arrival Delay']
features = df.drop(['Arrival Delay'] , axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)

# MSE

# R^2

In [47]:
model = LinearRegression().fit(features_train,target_train)
predicted_valid = model.predict(features_valid)
mse = mean_squared_error(target_valid,predicted_valid)
print(f"MSE = {mse}")
print(f"RMSE = {mse ** 0.5}")

MSE = 2129.82405285553
RMSE = 46.15001682400051


In [50]:
r2 = r2_score(target_valid,predicted_valid)
print(r2)

0.09710497146204955


In [53]:
for i in range(1,16,1):
    model = RandomForestRegressor(random_state=12345, max_depth=i)
    model.fit(features_train, target_train)
    train_score = model.score(features_train,target_train)
    valid_score = model.score(features_valid, target_valid)
    print(train_score, valid_score, i)

0.03815576840207502 0.03768017026017112 1
0.06921492812012076 0.06856965849959018 2
0.08932438614183691 0.08748649651553397 3
0.11093938161788508 0.10081946972118772 4
0.13077434926024079 0.11408509823727742 5
0.15573373775938515 0.12663922748019418 6
0.1867114143329418 0.13882629616679 7
0.22553804990743154 0.1505472428938046 8
0.27122804939304745 0.15858814679139244 9
0.32301140277147744 0.1624622902088042 10
0.3784551354930391 0.16447073084035702 11
0.43399599734071415 0.16455159363220306 12
0.49065430476831606 0.16155033528060292 13
0.5447308656763564 0.15832784317965753 14
0.5947879439607966 0.15480539457763154 15


In [56]:
%%time
model = RandomForestRegressor(n_estimators=100,  max_depth=12, random_state=12345)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
r2 = r2_score(target_valid,predicted_valid)
print(r2)


0.16455159363220306
CPU times: total: 18.5 s
Wall time: 18.6 s


In [62]:
def mae(target,predictions):
    error = 0
    for i in range(target.shape[0]):
        error += abs(target[i] - predictions[i])
    return error/target.shape[0]

In [63]:
target = pd.Series([-0.5, 2.1, 1.5, 0.3])
predictions = pd.Series([-0.6, 1.7, 1.6, 0.2])
print(mae(target, predictions))

0.17500000000000004


In [67]:
mae_formula = mean_absolute_error(target_valid, predicted_valid)  ## max_depth = 12
print(mae_formula)

25.47745895064548


In [68]:
predicted_valid1 = pd.Series(target_train.median(), index = target_valid.index) ## для медианы
mae_formula = mean_absolute_error(target_valid, predicted_valid1)
print(mae_formula)

27.22281548413595


In [69]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)

print(mean_absolute_error(target_valid, predictions_valid)) 

27.436250978085894


In [89]:
best_mae = 10000 
best_depth = 0
for i in range(10,100,10):
    model = RandomForestRegressor(n_estimators = i, max_depth= 11, random_state=12345)
    model.fit(features_train,target_train)
    predict = pd.Series()
    mae = mean_absolute_error(target_valid, predict)
    print(mae, i)

25.720627171113502 10
25.573148934738114 20
25.557784475337105 30
25.545920866528466 40
25.536218445193832 50
25.516633641960627 60
25.521262429946418 70
25.53200420061982 80
25.52338960838184 90


In [85]:
model = RandomForestRegressor(n_estimators = 80, max_depth= 11, random_state=12345)
model.fit(features_train,target_train)
predictions_train = model.predict(features_train)
predictions_valid  = model.predict(features_valid)
print(mean_absolute_error(target_train,predictions_train),mean_absolute_error(target_valid,predictions_valid))

23.358086544726298 25.53200420061982
